In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import xgboost as xgb
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ============ STEP 1: LOAD AND PREPARE DATA ============
print("Loading dataset...")
df = pd.read_csv(r'C:\Users\LENOVO\MachineLearningProhects\TimeSeriesForecasting_End_To_End\data\raw\electricity demand dataset.csv')  # Replace with your file path

# Create datetime index - Adjust based on your timestamp column
df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # If you have timestamp column
# Or create from the date columns you have:
# df['Timestamp'] = pd.date_range(start='2020-01-01', periods=len(df), freq='H')
df.dropna(inplace=True)
df = df.sort_values('Timestamp').reset_index(drop=True)
df.set_index('Timestamp', inplace=True)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(df.head())

# Extract target variable (Energy Demand)
ts_data = df['Demand'].values.astype(float)
print(f"\nTime series shape: {ts_data.shape}")
print(f"Missing values: {pd.isna(ts_data).sum()}")

Loading dataset...
Dataset shape: (43817, 8)
Date range: 2020-01-01 00:00:00 to 2024-12-31 00:00:00
            hour  dayofweek  month    year  dayofyear  Temperature   Humidity  \
Timestamp                                                                       
2020-01-01   0.0        2.0    1.0  2020.0        1.0     3.000000  61.288951   
2020-01-01  23.0        2.0    1.0  2020.0        1.0     3.000000  51.487121   
2020-01-01  22.0        2.0    1.0  2020.0        1.0     3.000000  56.571654   
2020-01-01  21.0        2.0    1.0  2020.0        1.0     3.000000  59.666195   
2020-01-01  20.0        2.0    1.0  2020.0        1.0     3.835712  44.737649   

                 Demand  
Timestamp                
2020-01-01  2457.119872  
2020-01-01  2622.193016  
2020-01-01  3213.228194  
2020-01-01  3890.171574  
2020-01-01  4424.292539  

Time series shape: (43817,)
Missing values: 0


In [3]:
df

,hour,dayofweek,month,year,dayofyear,Temperature,Humidity,Demand
Timestamp,,,,,,,,
2020-01-01,0.0,2.0,1.0,2020.0,1.0,3.000000,61.288951,2457.119872
2020-01-01,23.0,2.0,1.0,2020.0,1.0,3.000000,51.487121,2622.193016
2020-01-01,22.0,2.0,1.0,2020.0,1.0,3.000000,56.571654,3213.228194
2020-01-01,21.0,2.0,1.0,2020.0,1.0,3.000000,59.666195,3890.171574
2020-01-01,20.0,2.0,1.0,2020.0,1.0,3.835712,44.737649,4424.292539
...,...,...,...,...,...,...,...,...
2024-12-31,2.0,1.0,12.0,2024.0,366.0,3.000000,55.076004,2431.538942
2024-12-31,1.0,1.0,12.0,2024.0,366.0,3.000000,79.536292,2654.477513
2024-12-31,0.0,1.0,12.0,2024.0,366.0,3.000000,74.124312,2722.480185


In [4]:
ts_data

array([2457.119872, 2622.193016, 3213.228194, ..., 2722.480185,
       4813.689951, 3219.023339], shape=(43817,))